<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 03</h1>

Learning objectives:

1. Examine real data 
<p>

2. Use `lm()` to run ordinary least squares, regressing $y$ on an $x$ where $x$ is a binary 0/1 indicator of group status
<ul>
  <li> Members of the group get a 1
  <li> Others (the "default category") get a 0
</ul>

<p>
    
3. Under these conditions, `lm()` shows you
<ul>
    <li>$y = \alpha + \beta x$
    <li>The average $y$ for the default category = constant term $\alpha$
    <li>The difference in average $y$ between groups = slope term $\beta$
    <li>Standard errors for each of these, in addition to point estimates
</ul>
    
<h3>For example</h3>

Suppose $y$ is itself a binary 0/1 measure of having heart problems: either an individual reports having been diagnosed with heart problems or does not. Suppose also that $x$ measures identifying as Black but not Hispanic, which is a common definition of a distinct racial and ethnic group in the U.S.

Then running `lm()` to estimate `y ~ x` gives us the average rate of heart problems among non-Black people, which is the constant term $\alpha$; and the difference in average rates of heart problems between Black people and non-Black people, which is the slope term $\beta$. Also, the standard error of $\beta$ helps us understand whether that difference is statistically significant.

(Heart problems are defined in the U.S. Health and Retirement Study as: "a heart attack, coronary heart disease, angina, congestive heart failure, or other heart problems.")

There are a lot of outcomes ($y$) that we would like to compare across groups defined in various ways. In <i>Mastering Metrics</i>, we started by comparing people across health insurance status in Chapter 1. Then later in that chapter, we see just how different the story becomes when we examine the RAND Health Insurance Experiment, when treatment groups were randomly assigned to different health insurance plans.

As we will see, when you can't run a randomized controlled trial (RCT) like that, you have to figure out what to do next. For now, let's explore some characteristics of respondents to the 14th wave of the U.S. [Health and Retirement Study (HRS)](https://hrs.isr.umich.edu/welcome-health-and-retirement-study), conducted mostly in 2018, before the COVID-19 pandemic. The HRS is a panel survey, with biennial waves, of Americans aged 50 and over.

Let's load up <b>haven</b> and <b>tidyverse</b>

In [ ]:
library(haven)
library(tidyverse)

I have prepared an extract from the RAND version of the HRS file, and I have created some additional helpful variables. Here is the extract:

In [ ]:
hrs_heart <- read_dta("data/hrs_heart.dta")

In [ ]:
head(hrs_heart)

There is a lot here. The way RAND names the variables is `RwVARNAME` where "w" is either the number of the wave, or the letter "a" if the measure should be the same across all waves. For example, above age 50, years of education usually never changes, so that variable is named `RAEDYRS`. (Note that here, all variables are in lowercase.)

Let us example the measure in wave 14 of the binary response to the question, "Has has a doctor (ever) told you that you had a heart attack, coronary heart disease, angina, congestive heart failure, or other heart problems?" This variable is `r14hearte`.

I have created a quadrichotomous (is that a word?) variable called `raraceth` that combines race and ethnicity information into four broad, useful, and common categories: 

(1) White non-Hispanic people

(2) Black non-Hispanic people

(3) Hispanic people

(4) Non-Hispanic people of other race

Especially in relatively small samples, this breakdown is helpful. It certainly doesn't cover all variation in lived experiences, but it's a good balance between parsimony and breadth.

<h2>Look at differences in average heart problems</h2>

There are lots of ways of coding this. Here is one that is more elegant, which ChatGPT suggested to me:

In [ ]:
# Use tidyverse "pipes." Thanks, ChatGPT
# 
# Look at white non-Hispanic people, where raraceth == 1
avg_hearte_wnh <- hrs_heart %>%  
    filter(raraceth == 1) %>% 
    summarize(average = mean(r14hearte, na.rm = TRUE))
avg_hearte_wnh

In [ ]:
# Look at Black non-Hispanic people where raraceth == 2 
avg_hearte_bnh <- hrs_heart %>%  
    filter(raraceth == 2) %>% 
    summarize(average = mean(r14hearte, na.rm = TRUE))
avg_hearte_bnh

In [ ]:
# Difference between these averages
# = excess rate of heart problems among Black people
avg_hearte_bnh - avg_hearte_wnh

In words, the number above is the excess average rate of heart problems among Black non-Hispanic people in this sample compared to white non-Hispanic people. It is equal to $-0.062$, meaning that Black people in the sample are 6.2 percentage points less likely to report ever having heart problems.

Here is another way to proceed. You can also use `filter()` from `dplyr` to cut the data frame up into new data frames that condition on something, which here is race/ethnicity:

In [ ]:
hrs_heart_wnh <- filter(hrs_heart, raraceth == 1)
hrs_heart_bnh <- filter(hrs_heart, raraceth == 2)
hrs_heart_wbnh <- filter(hrs_heart, raraceth == 1 | raraceth == 2)

In [ ]:
summary(hrs_heart_wnh)

Before we whittle this down a little, let's also look now at Black non-Hispanic people:

In [ ]:
summary(hrs_heart_bnh)

Look at the "Mean" entries for WNH people and the variables `r14hearte` and `r14agey_m`, the latter of which is the respondent's age in years. (The code below might help make this clearer.) What do you see?

In [ ]:
summary(hrs_heart_wnh$r14hearte)
summary(hrs_heart_wnh$r14agey_m)

Let's compute the same for BNH people:

In [ ]:
summary(hrs_heart_bnh$r14hearte)
summary(hrs_heart_bnh$r14agey_m)

What do you see? Compare the means of these two variables across WNH and BNH people in a few words or sentences in the markdown field below:

<h2>Regression with indicator variables gives you conditional averages</h2>

Hold on to your hats.

It turns out that a linear regression of $y$ on an $x$ that is dichotomous (equal to 0 or 1) will tell us the average $y$ for the group whose $x = 0$, that's the constant term $\alpha$; and it will tell us the difference between that group's average $y$ and the other group's (indicated by $x = 1$).

In other words, when we run
$$y = \alpha + \beta x + \epsilon$$

when $x$ is a dichtomous "indicator variable" (sometimes called "dummy variable"), $\alpha$ is the average $y$ for the $x = 0$ group, and $\alpha + \beta$ is the average $y$ for the $x = 1$ group.

Observe. Recall that `hrs_heart_wbnh` is the data frame that includes both WNH and BNH people. Let's run this regression:
$$heart = \alpha + \beta \cdot BNH + \epsilon$$

where `heart` is shorthand for `r14hearte` and `BNH` is the indicator variable for identifying as a Black non-Hispanic person, `rablacknonh`.

In [ ]:
heartreg <- lm(r14hearte ~ rablacknonh, data = hrs_heart_wbnh)
summary(heartreg)

Interesting. Consider this arithmetic from the averages of `r14hearte` among BNH people and WNH people shown earlier above:

In [ ]:
0.2226 - 0.2847

Do you see similar numbers in the arithmetic and in the regression output? Where? Write some thoughts in the markdown field below.

Should we be worried that WNH people are on average about 5 years older than BNH people in these data? If we were, what could we do about it? Maybe try this:
$$heart = \alpha + \beta \cdot BNH + \delta \cdot age + \epsilon$$

In [ ]:
heartreg_age <- lm(r14hearte ~ rablacknonh + r14agey_m, data = hrs_heart_wbnh)
summary(heartreg_age)

There is certainly a lot to see here. What happened to the coefficient on `BNH`, shown here in the "Estimate" column and on the `rablacknonh` row? Look at far right. Do you see any asterisks in that row now?

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>